https://archive.ics.uci.edu/ml/machine-learning-databases/ionosphere/ionosphere.data

In [4]:
#imports
from torch.utils.data import Dataset
from pandas import read_csv
from sklearn.preprocessing import LabelEncoder
from torch.nn import Module

### Dataset definition
PyTorch provides the Dataset class that you can extend and customize to load your dataset.
For example, the constructor of your dataset object can load your data file (e.g. a CSV file). You can then override the __len__() function that can be used to get the length of the dataset (number of rows or samples), and the __getitem__() function that is used to get a specific sample by index.

The random_split() function can be used to split a dataset into train and test sets. Here we are dividing 67 train and 33 test data.

##### Current CSVDataset
label is the last one in the csv. It is a string represented as 'g': good and 'b': bad.
We use LabelEncoder to encode string labels to 0 and 1's.
We convert X and y's as floats.

Why self.y.reshape?




In [2]:
class CSVDataset(Dataset):
    def __init__(self,path):
        dataframe = read_csv(path, header = None)
        
        self.X = dataframe.values[:,:-1]
        self.y = dataframe.values[:,-1]
        
        self.X = self.X.astype('float32')
        self.y = LabelEncoder().fit_transform(self.y)
        self.y = self.y.astype('float32')
        print(shape(self.y))
        self.y = self.y.reshape(len(self.y),1)
    
    def __len__(self):
        return len(self.X)
    
    def __getItem__(self,idx):
        return [self.X[idx],self.y[idx]]
    
    def get_split_data(self,n_train = 0.67):
        train_len = len(self.X)*n_train
        test_len = len(self.X) - train_len
        return random_split(self,[train_len,test_len])
        
        

### Model definition
Linear 

In [5]:
class MLP(Module):
    def __init__(self, inputs):
        super(MLP,self).__init__()
        
        self.hidden1 = Linear(inputs,10)
        kaiming_uniform_(self.hidden1.weight,nonlinearity = 'relu')
        self.act1 = ReLU()
        
        self.hidden2 = Linear(10,8)
        kaiming_uniform_(self.hidden2.weight, nonlinearity = 'relu')
        self.act2 = ReLU()
        
        self.hidden3 = Linear(8,1)
        xavier_uniform_(self.hidden3.weight)
        self.act3 = Sigmoid()
        
    def forward(self,X):
        X = self.hidden1(X)
        X = self.act1(X)
        
        X = self.hidden2(X)
        X = self.act2(X)
        
        X = self.hidden3(X)
        X = self.act3(X)
        return X